In [30]:
import torch
results = torch.load('results/res_test_none.pt')
# results = torch.load('results/res_test_ma_w100_l5_wd10e-02.pt')
# results.keys()

/tmp/ipykernel_2752431/2491411562.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  results = torch.load('results/res_test_none.pt')


In [35]:
weights = results['net']

In [8]:
import torch
from torch import nn

from dattri.algorithm.influence_function import IFAttributor
from dattri.benchmark.datasets.mnist import train_mnist_lr, create_mnist_dataset
from dattri.func.utils import flatten_func
from dattri.benchmark.utils import SubsetSampler

In [9]:
def multiplication_mod_p_data(p, eq_token, op_token):
    """x◦y = x/y (mod p) for 0 ≤ x < p, 0 < y < p
    """
    x = torch.arange(p)
    y = torch.arange(1, p)
    x, y = torch.cartesian_prod(x, y).T

    eq = torch.ones_like(x) * eq_token
    op = torch.ones_like(x) * op_token
    result = x * y % p

    # "All of our experiments used a small transformer trained on datasets of
    # equations of the form a◦b = c, where each of “a”, “◦”, “b”, “=”, and “c”
    # is a seperate token"
    return torch.stack([x, op, y, eq, result])

from argparse import ArgumentParser
parser = ArgumentParser()
parser.add_argument("--p", type=int, default=97)
args = parser.parse_args('')


eq_token = args.p
op_token = args.p + 1

data = multiplication_mod_p_data(args.p, eq_token, op_token)

train_idx, valid_idx = torch.randperm(data.shape[1]).split(data.shape[1] // 2)
train_data, valid_data = data[:, train_idx], data[:, valid_idx]


139

In [24]:
# dataset_train, dataset_test = create_mnist_dataset("./data")


In [25]:
# dataset_train, dataset_test = create_mnist_dataset("./data")

# Algorithmic data
dataset_train = train_data.T
dataset_test = valid_data.T

train_loader = torch.utils.data.DataLoader(
    dataset_train,
    batch_size=64,
    sampler=SubsetSampler(range(1000)),
)
test_loader = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=64,
    sampler=SubsetSampler(range(100)),
)


In [27]:
next(iter(train_loader))

torch.Size([64, 5])

In [28]:
from main import Decoder
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")

model = Decoder(
    dim=128, num_layers=2, num_heads=4, num_tokens=args.p + 2, seq_len=5
).to(device)


In [38]:
model.load_state_dict(weights[0])

<All keys matched successfully>

In [40]:
# model = train_mnist_lr(train_loader)

@flatten_func(model)
def f(params, data_target_pair):
    print(data_target_pair)
    x, y = data_target_pair
    loss = nn.CrossEntropyLoss()
    yhat = torch.func.functional_call(model, params, x)
    return loss(yhat, y)

model_params = {k: p for k, p in model.named_parameters() if p.requires_grad}
attributor = IFAttributor(
    target_func=f,
    params=model_params,
    ihvp_solver="cg",
    ihvp_kwargs={"max_iter": 10, "regularization": 1e-2},
)

attributor.cache(train_loader)
score = attributor.attribute(train_loader, test_loader)

(GradTrackingTensor(lvl=2, value=
    BatchedTensor(lvl=1, bdim=1, value=
        tensor([[10, 98,  8, 97, 80]])
    )
), GradTrackingTensor(lvl=2, value=
    BatchedTensor(lvl=1, bdim=1, value=
        tensor([[12, 98, 55, 97, 78]])
    )
), GradTrackingTensor(lvl=2, value=
    BatchedTensor(lvl=1, bdim=1, value=
        tensor([[ 9, 98,  7, 97, 63]])
    )
), GradTrackingTensor(lvl=2, value=
    BatchedTensor(lvl=1, bdim=1, value=
        tensor([[77, 98, 10, 97, 91]])
    )
), GradTrackingTensor(lvl=2, value=
    BatchedTensor(lvl=1, bdim=1, value=
        tensor([[65, 98,  9, 97,  3]])
    )
), GradTrackingTensor(lvl=2, value=
    BatchedTensor(lvl=1, bdim=1, value=
        tensor([[29, 98, 12, 97, 57]])
    )
), GradTrackingTensor(lvl=2, value=
    BatchedTensor(lvl=1, bdim=1, value=
        tensor([[33, 98, 29, 97, 84]])
    )
), GradTrackingTensor(lvl=2, value=
    BatchedTensor(lvl=1, bdim=1, value=
        tensor([[26, 98, 57, 97, 27]])
    )
), GradTrackingTensor(lvl=2, value

ValueError: too many values to unpack (expected 2)